In [1]:
# CONECTANDO AO GOOGLE DRIVE (GD)
from google.colab import drive

drive.mount('/content/drive')

gd_folder = '/content/drive/MyDrive/xco2_2020_julho/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
# CONECTANDO AO GOOGLE CLOUD STORAGE (GCS)
PROJECT = 'mapbiomas'

from google.colab import auth
auth.authenticate_user()

!gcloud config set project {PROJECT}

gcs_folder = 'gs://shared-development-storage/AUXILIARES/MAPBIOMAS-AR/OCO2/'

Updated property [core/project].


In [3]:
# CONECTANDO AO GOOGLE EARTH ENGINE (GEE)
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=PofoMpO7n_GltpjNE4y7yYrmia7D95Yg_nzQY0yf7tY&tc=8ORMRVUSr_j16dSbEk--GEw0FUVSMr95OJeSc73Nz2I&cc=NsLvvZQQFhxPU0hVBeb1_L5QMH2S29BKvToqw0-RlGE

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWgugSE20Fyrb0tShradU_WSYDV5EGfjfpga36vt7Houln-rcTqpn9g

Successfully saved authorization token.


In [4]:
#  INSTALANDO BIBLIOTECAS INEXISTENTES NO GOOGLE COLLAB (GC)
!pip install rioxarray
!pip install geopandas

In [5]:
# IMPORTANDO BIBLIOTECAS
import xarray as xr
import rioxarray as rio
import geopandas as gpd
import pandas as pd
import os
import numpy as np
from osgeo import gdal
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, Point, MultiPolygon

In [16]:
# CONSULTANDO ENDEREÇO DOS ARQUIVOS ".nc4" NA PASTA DO GD
nc4s_files = os.listdir(gd_folder)
nc4s_files = list(filter(lambda f: f.find(".nc4") >= 0, nc4s_files))[2:]

nc4s_files

['oco2_LtCO2_200728_B10206Ar_210920055619s.nc4',
 'oco2_LtCO2_200726_B10206Ar_210920055119s.nc4',
 'oco2_LtCO2_200725_B10206Ar_210920054827s.nc4',
 'oco2_LtCO2_200724_B10206Ar_210920054645s.nc4',
 'oco2_LtCO2_200723_B10206Ar_210920054327s.nc4',
 'oco2_LtCO2_200722_B10206Ar_210920054144s.nc4',
 'oco2_LtCO2_200720_B10206Ar_210920053632s.nc4',
 'oco2_LtCO2_200719_B10206Ar_210920053239s.nc4',
 'oco2_LtCO2_200718_B10206Ar_210920053127s.nc4',
 'oco2_LtCO2_200717_B10206Ar_210920052736s.nc4',
 'oco2_LtCO2_200716_B10206Ar_210920052633s.nc4',
 'oco2_LtCO2_200714_B10206Ar_210920052242s.nc4',
 'oco2_LtCO2_200713_B10206Ar_210920051846s.nc4',
 'oco2_LtCO2_200711_B10206Ar_210920051358s.nc4',
 'oco2_LtCO2_200709_B10206Ar_210920050853s.nc4',
 'oco2_LtCO2_200708_B10206Ar_210920050824s.nc4',
 'oco2_LtCO2_200706_B10206Ar_210920050303s.nc4',
 'oco2_LtCO2_200704_B10206Ar_210920045830s.nc4',
 'oco2_LtCO2_200703_B10206Ar_210920045226s.nc4',
 'oco2_LtCO2_200702_B10206Ar_210920045208s.nc4',
 'oco2_LtCO2_200701_

In [ ]:
# LOOP DE CONVERSÃO DOS ARQUIVOS .NC4 EM SHAPEFILE E UPLOAD PARA O GEE PASSANDO PELO GCS
print('loop start')
for filename in nc4s_files:

  nc_file = xr.open_dataset(gd_folder + filename)
  xco2 = nc_file[[
    'longitude',
    'latitude',
    'xco2',
    'time',
    'vertex_longitude',
    'vertex_latitude'
  ]]

  xco2.rio.set_crs("epsg:4326")

  df = xco2.to_dataframe()

  df = df.reset_index()

  df['vertices'] = df['vertices'].astype(str)

  dfp = pd.pivot_table(df,
                     index=['sounding_id', 'time', 'latitude', 'longitude', 'xco2'],
                     columns='vertices',
                     values=['vertex_longitude', 'vertex_latitude'])

  dfp = dfp.reset_index()

  dfp.columns = dfp.columns.map('_'.join).str.strip('_')

  dfp['geometry'] = dfp.apply(
      lambda s: Polygon(shell=[
          Point(s.vertex_longitude_1, s.vertex_latitude_1),
          Point(s.vertex_longitude_2, s.vertex_latitude_2),
          Point(s.vertex_longitude_3, s.vertex_latitude_3),
          Point(s.vertex_longitude_4, s.vertex_latitude_4),
          Point(s.vertex_longitude_1, s.vertex_latitude_1)
      ]),
      axis=1
  )

  dfp.pop('vertex_longitude_1')
  dfp.pop('vertex_longitude_2')
  dfp.pop('vertex_longitude_3')
  dfp.pop('vertex_longitude_4')
  dfp.pop('vertex_latitude_1')
  dfp.pop('vertex_latitude_2')
  dfp.pop('vertex_latitude_3')
  dfp.pop('vertex_latitude_4')
  dfp.pop('longitude')
  dfp.pop('latitude')
  
  dfp['time'] = dfp['time'].astype(str)
  dfp['file_id'] = filename

  gdf = gpd.GeoDataFrame(dfp, geometry='geometry', crs="EPSG:4326")
  gdf['area'] = gdf['geometry'].area / 10**6
  gdf['id'] = range(1, len(gdf) + 1)

  file_shp = gd_folder + 'scripts/shp-v3/' + filename[:-4] +'-v1.shp'

  gdf.to_file(file_shp)

  os.system('gsutil cp {0} {1}shps/{2}-v1.shp'.format(file_shp, gcs_folder, filename[:-4]))
  os.system('gsutil cp {0} {1}shps/{2}-v1.dbf'.format(file_shp, gcs_folder, filename[:-4]))
  os.system('gsutil cp {0} {1}shps/{2}-v1.shx'.format(file_shp, gcs_folder, filename[:-4]))
  os.system('gsutil cp {0} {1}shps/{2}-v1.prj'.format(file_shp, gcs_folder, filename[:-4]))

  # os.system('gsutil cp {0} {1}nc4s/{2}'.format(gd_folder + filename, gcs_folder, filename))

  
  asset_id = 'projects/mapbiomas-workspace/AR/XCO2-VECTORS/'
  # os.system('earthengine upload table --asset_id={0} {1}'.format(asset_id + filename[:-4] + '-v1', gcs_folder + 'shps/' + filename[:-4] + '-v1.shp'))

  print('nc4 file: ',filename,' - ',nc4s_files.index(filename) + 1, ' de ', len(nc4s_files))

  # cada arquivo demora um pouco mais de 3 minutos para concluir o loop

# 23 arquivos levaram um pouco menos de 1 hora e meia
print('end loop')

loop start


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:58: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:63: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.


nc4 file:  oco2_LtCO2_200728_B10206Ar_210920055619s.nc4  -  1  de  21


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:58: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:63: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.


nc4 file:  oco2_LtCO2_200726_B10206Ar_210920055119s.nc4  -  2  de  21


...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:58: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:63: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.


nc4 file:  oco2_LtCO2_200719_B10206Ar_210920053239s.nc4  -  8  de  21


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:58: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:63: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.


nc4 file:  oco2_LtCO2_200718_B10206Ar_210920053127s.nc4  -  9  de  21


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:58: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:63: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
